In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

def parse_branch_info(a_tag, base_url):
    """
    Từ một thẻ <a> chứa link Google Maps, leo lên các parent
    để tìm thẻ <p> đầu tiên (chứa tên chi nhánh).
    """
    # Link tuyệt đối đến Google Maps
    map_link = urljoin(base_url, a_tag['href'])
    # Lấy địa chỉ (text bên trong <a>)
    address = a_tag.get_text(strip=True)

    # Leo qua các parent để tìm <p> đầu tiên
    name = ''
    for parent in a_tag.parents:
        p = parent.find('p')
        if p:
            name = p.get_text(strip=True)
            break

    return {
        'branch_name': name,
        'address': address,
        'map_link': map_link
    }

def scrape_coopmart_branches(base_url, total_pages=13):
    results = []
    for page in range(1, total_pages + 1):
        # Xây URL phân trang
        url = base_url if page == 1 else f"{base_url}?page={page}"
        print(f"Đang tải trang {page}: {url}")
        resp = requests.get(url)
        if resp.status_code != 200:
            print(f"  → Lỗi {resp.status_code}, bỏ qua.")
            continue

        soup = BeautifulSoup(resp.content, 'html.parser')
        # Tìm tất cả <a> dẫn đến Google Maps
        a_map = soup.find_all(
            'a',
            href=lambda h: h and (
                'maps.app.goo.gl' in h or 
                'google.com/maps' in h or 
                'maps.google.com' in h
            )
        )
        print(f"  → Tìm được {len(a_map)} link Google Maps")
        for a in a_map:
            info = parse_branch_info(a, base_url)
            results.append(info)

    # Chuyển thành DataFrame và loại bỏ trùng lặp
    df = pd.DataFrame(results).drop_duplicates().reset_index(drop=True)
    return df

if __name__ == '__main__':
    BASE_URL = 'https://co-opmart.com.vn/he-thong-sieu-thi'
    df_branches = scrape_coopmart_branches(BASE_URL, total_pages=13)
    print("\nKết quả:")
    print(df_branches)
    # Nếu cần lưu ra file CSV:
    # df_branches.to_csv('coopmart_branches.csv', index=False)


Đang tải trang 1: https://co-opmart.com.vn/he-thong-sieu-thi
  → Tìm được 11 link Google Maps
Đang tải trang 2: https://co-opmart.com.vn/he-thong-sieu-thi?page=2
  → Tìm được 11 link Google Maps
Đang tải trang 3: https://co-opmart.com.vn/he-thong-sieu-thi?page=3
  → Tìm được 11 link Google Maps
Đang tải trang 4: https://co-opmart.com.vn/he-thong-sieu-thi?page=4
  → Tìm được 11 link Google Maps
Đang tải trang 5: https://co-opmart.com.vn/he-thong-sieu-thi?page=5
  → Tìm được 11 link Google Maps
Đang tải trang 6: https://co-opmart.com.vn/he-thong-sieu-thi?page=6
  → Tìm được 11 link Google Maps
Đang tải trang 7: https://co-opmart.com.vn/he-thong-sieu-thi?page=7
  → Tìm được 11 link Google Maps
Đang tải trang 8: https://co-opmart.com.vn/he-thong-sieu-thi?page=8
  → Tìm được 11 link Google Maps
Đang tải trang 9: https://co-opmart.com.vn/he-thong-sieu-thi?page=9
  → Tìm được 11 link Google Maps
Đang tải trang 10: https://co-opmart.com.vn/he-thong-sieu-thi?page=10
  → Tìm được 11 link Google 

In [2]:
df_branches

,branch_name,address,map_link
0,CO.OPMART CỐNG QUỲNH,"189C Cống Quỳnh, Phường Nguyễn Cư Trinh, Quận 1",https://maps.app.goo.gl/seAwusqkxfmjgzRS9
1,CO.OPMART BÌNH TRIỆU,"68/1 Quốc Lộ 13, Phường Hiệp Bình Chánh, Thành...",https://maps.app.goo.gl/hDkYbGM4UX5K6gD96
2,CO.OPMART VŨNG TÀU,"36 Nguyễn Thái Học, Phường 7, Tp. Vũng Tàu, Tỉ...",https://maps.app.goo.gl/DR9VtNZCrNw4djch8
3,CO.OPMART BÀ RỊA,"Số 6, Nguyễn Hữu Thọ, Khu Phố 2, Phường Phước ...",https://maps.app.goo.gl/Tu7Kv5qXy3BVf7TA8
4,CO.OPMART BIÊN HÒA,"121 Phạm Văn Thuận, Phường Tân Tiến, Biên Hòa...",https://maps.app.goo.gl/SNuvMhnCh2Zg5GiPA
...,...,...,...
118,CO.OPMART CHƯ SÊ,"912, Hùng Vương, Tdp4, Tt Chư Sê, Huyện Chư Sê...",https://maps.app.goo.gl/k7Js9JzuD8CZWdGU9
119,CO.OPMART CƯMGAR,"Thửa Đất 11, Tờ Bản Đồ Số 31, Tổ Dân Phố 2 Đườ...",https://maps.app.goo.gl/Nn7ai8JxA9Ygjvq77
120,CO.OPMART ĐAKNONG,Đường Huỳnh Thúc Kháng – Tổ Dân Phố 1 – Phường...,https://maps.app.goo.gl/29xJ3EaseZ1mLWBt7
121,CO.OPMART KON TUM,"205 B Lê Hồng Phong, P.Quyết Thắng, Tp Kon Tum...",https://maps.app.goo.gl/LTs2CsKE7v1d6EkFA


In [3]:
df_branches.to_csv('coopmart_branches.csv', index=False)